# Capstone Project - Cryptocurrencies

### Phase 2: Data Collection

In [5]:
import numpy as np
import pandas as pd
import pandas_ta as ta
pd.set_option('display.max_columns', None)

import pandas_datareader.data as web
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import datetime as dt
import os
import tensorflow as tf
import time
import itertools

from tqdm import tqdm # a progress bar

plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

### Goal: Using Data Science & ML/DL Methods to evaluate Crypto Prices

- Evaluating methods/indicators used in Technical Analysis in trading
- Generate a scoring function (with a classification models) that ranks (& adjusts parameters) & implements each indicator.
- Use this on a standard ARIMA model to visualize 5 day (120 hour) projections
- Backtesting on historical data (hourly)

### Note on how crypto exchanges function:

- Unlike stocks, but like forex, run 24/7
- Prices open at midnight & close just before (minute, second?) - high.low within 24 hour period
- Unlike fiat currency markets, where prices usually don’t move by more than 1% each day, cryptocurrency prices can almost double (or halve) overnight!
- This makes them more suitable for short-term trading (e.g. daily or even less) if you want to visualize tangible changes. If you have the resources & knowledge, you can try out HFT.
- Cryptocurrencies are (usually) decentralised, meaning it is not issued or backed by a central authority such as a central bank or government. Instead, cryptocurrencies run across a network of computers.
- Cryptocurrencies bear little resemblance to other asset classes because they are intangible and extremely volatile. They are mainly used by traders for speculating on rises and falls in value.

### Get data in OHLCV format

### Save each cryptocurrency (with the right amount of technical indicators - all same) to its own csv.

- Target ~24,000 rows per currency (hourly, 1000 days) min??
- Find a way to compare the models
- Regressor model first for each cryptocurrency (ARIMA & related)

- Then consider moving to the classifier evaluation in example from site below:
- https://nycdatascience.com/blog/student-works/bitcoin-signal-analysis/

# Phase 2: Data Collection, Cleaning & Feature Engineering

## Using intra-day data with an API

#### Using Coinbase Pro API for Intraday data (hourly)

- May be possible to get real time data too (check API limitations)
- Will need looping function to do over a few years

- **By default, each API key or app is rate limited at 10,000 requests per hour.**

- See https://algotrading101.com/learn/coinbase-api-guide/

In [2]:
# Will need new API for intra day data
# Hide API keys - saved locally to machine (MBP)


import cbpro

cbpro_nickname = os.environ.get("CBPRO_NICKNAME")

cbpro_api_key = os.environ.get("CBPRO_API")

cbpro_paraphrase = os.environ.get("CBPRO_PHRASE")


api_url = 'https://api.pro.coinbase.com/'


auth = cbpro.AuthenticatedClient(cbpro_nickname,
                            cbpro_api_key,
                            cbpro_paraphrase, api_url)

In [3]:
# Choose Start & End Date (YYYY-MM-DD)



# Works daily and weekly but not more?? (max 9 days or 200 candles)

# start_date = "2021-01-24"# Depending on the crypto used, will have uneven observations (due to age of the currency)

# end_date = "2021-01-30"

# timeframe = 3600  # in seconds (3600 hourly)

Signature:
`auth.get_product_historic_rates(
    product_id,
    start=None,
    end=None,
    granularity=None)`

- Docstring:
Historic rates for a product.

Rates are returned in grouped buckets based on requested
`granularity`. If start, end, and granularity aren't provided,
the exchange will assume some (currently unknown) default values.

Historical rate data may be incomplete. No data is published for
intervals where there are no ticks.

**Caution**: Historical rates should not be polled frequently.
If you need real-time information, use the trade and book
endpoints along with the websocket feed.

**The maximum number of data points for a single request is 200
candles. If your selection of start/end time and granularity
will result in more than 200 data points, your request will be
rejected. If you wish to retrieve fine granularity data over a
larger time range, you will need to make multiple requests with
new start/end ranges.**

Args:
    product_id (str): Product
    start (Optional[str]): Start time in ISO 8601
    end (Optional[str]): End time in ISO 8601
    granularity (Optional[int]): Desired time slice in seconds

Returns:
    list: Historic candle data. Example:
        [
            [ time, low, high, open, close, volume ],
            [ 1415398768, 0.32, 4.2, 0.35, 4.2, 12.3 ],
            ...
        ]
      
- File:      ~/opt/anaconda3/lib/python3.8/site-packages/cbpro/public_client.py
- Type:      method

### Function to iterate & scrape over date range for this annoying API that limits to 200 candles per request

- Weekly data will give hourly candles (145)
- Intending to go back roughly 5 years as of January 2021
- Looping function to iterate and 'scrape' historical hourly crypto data

In [4]:
# As a function:
# For use of Coinbase Pro API, day can't be past ~9 as it limits the max no of candlesticks
# you can retrieve with one request. 7 a good number as it's weekly

def date_range_thing(day=7, delta=38):
    """ Returns a list containing tuples of successive chosen dates (separated by day)
        for an API or scraper to iterate through in terms of a given date range,
        beginning from today's date. Delta will alter the total time frame. """
    
    # Counting date range
    # The size of each step in days and putting them into a list
    
    day_delta = dt.timedelta(days=day)

    end_date = dt.date.today()
    start_date = end_date - (delta * day_delta) # Trial & error - leads to 3/1/2016 - roughly 5 years of weekly data

    date_lst = []
    for i in range((end_date - start_date).days):
        date_lst.append(end_date - i*day_delta)
      
    # Convert datetime to str
    
    date_lst_str = []
    for date in date_lst:
        date_lst_str.append(date.strftime("%Y-%m-%d"))

    # Functions to put consecutive weekly dates into a tuple, which will be the start date & end date to iterate through
    # Fix this section so you won't get duplicate dates (or remember to drop duplicates on 'Time')
    
    def advance(iterator, step):
        next(itertools.islice(iterator, step, step), None)


    def tuplewize(iterable, size):
        iterators = itertools.tee(iterable, size)
        for position, iterator in enumerate(iterators):
            advance(iterator, position)
        return zip(*iterators)
    
    # List output of tuples, in str format
    # Note the duplicated days, fix or drop duplicates
    
    date_pairs = []
    for t in tuplewize(date_lst_str, 2):
        date_pairs.append(t)
    
    return date_pairs

### Function to retrieve historical data

In [5]:
import time
from tqdm import tqdm # a progress bar

In [6]:
# Test for weeks per delta
# 1 = 6 weeks, but 2 = 13 weeks?
# This adds 7 weeks when delta > 1

# trial & error leads to 5 years with delta = 38

# date_range_thing(7, delta=38)

In [7]:
def get_crypto_df(ticker, day=7, delta=38):
    """ Gets a dataframe of hourly crypto index data over the past 5 years with delta=38.
        Each unit increase in delta adds n days (from day argument) to timeframe.
        Add ticker in the right format e.g. 'BTC-USD' """
    
    # date range function to get dates
    date_range = date_range_thing(day, delta)
    
    timeframe = 3600 # In seconds, this returns hourly data
    
    end_dates = []
    start_dates = []
    
    for tup in tqdm(date_range):
        
        # Import time & random sleep function to not stress API
        time.sleep(np.abs(np.random.normal(loc=2, scale=0.5)))
        
        
        end_date = tup[0]
        start_date = tup[1]
        
        # append all possible date range from delta
        
        end_dates.append(end_date)
        start_dates.append(start_date)
    

        crypto_lst_pre = []
    
        for (end_d, start_d) in zip(end_dates, start_dates): 
        
            crypto_lst = auth.get_product_historic_rates(ticker, start=start_d, 
                                                    end=end_d, granularity=timeframe)
            
            # list of lists of output ready for df conversion
            crypto_lst_pre.append(crypto_lst)
            crypto_lst_main  = [val for sublist in crypto_lst_pre for val in sublist] 
            
            
            # Note formatting returned by API
            crypto_df = pd.DataFrame(crypto_lst_main, columns=['time', 'low', 'high', 'open', 'close', 'volume'])
            crypto_df['time'] = pd.to_datetime(crypto_df['time'], unit='s')


            # Remove duplicates for double 'time' observations made by date_range function
            # reorder cols to be OHLCV
            
            crypto_df = crypto_df.drop_duplicates(subset=['time'])
            crypto_df = crypto_df.reindex(columns=['time', 'open', 'high', 'low', 'close', 'volume'])
            
            crypto_df.index = crypto_df['time']
            crypto_df.index = pd.to_datetime(crypto_df.index)
            crypto_df.sort_index(inplace=True)
            crypto_df.drop(columns=['time'], inplace=True)
    
    return crypto_df

## Test functions

- Get a smaller sample dataset to experiment with

In [8]:
btcusd_test = get_crypto_df('BTC-USD', delta=1)

100%|██████████| 6/6 [00:17<00:00,  2.96s/it]


In [9]:
btcusd_test

,open,high,low,close,volume
time,,,,,
2020-12-26 00:00:00,24704.71,24770.30,24515.00,24600.00,571.509687
2020-12-26 01:00:00,24595.01,24900.00,24490.01,24895.46,732.313376
2020-12-26 02:00:00,24895.46,24967.38,24764.32,24832.71,969.940421
2020-12-26 03:00:00,24832.71,25000.00,24820.94,24893.03,1202.206191
2020-12-26 04:00:00,24893.03,24923.93,24750.00,24808.46,659.335643
...,...,...,...,...,...
2021-02-05 20:00:00,37658.06,37819.30,37527.33,37779.42,442.545004
2021-02-05 21:00:00,37772.30,37936.62,37716.00,37860.69,464.757661
2021-02-05 22:00:00,37860.68,38003.71,37720.00,37832.66,577.271040


In [10]:
btcusd_test.describe()

,open,high,low,close,volume
count,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000
mean,33675.762151,34000.029822,33306.742587,33689.793826,1398.405413
std,3782.537268,3827.323228,3724.276025,3775.334723,1151.263681
min,24595.010000,24770.300000,24490.010000,24600.000000,167.219444
25%,31592.530000,31942.180000,31174.560000,31600.000000,714.872815
50%,33847.180000,34233.000000,33533.000000,33866.420000,1072.052909
75%,36487.640000,36793.730000,36098.480000,36514.920000,1686.096522
max,41520.600000,41986.370000,41001.010000,41520.650000,10151.749811


We are interested in 4-5 categories of technical indicators:

- Momentum
- Volume
- Volitality
- Trend
- Other notable indicators from the other categories

## Method: Technical Analysis Library
- See:
https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html
- Github repo:
https://github.com/bukosabino/ta

In [21]:
from ta import add_all_ta_features # This auto calculates your TI columns
from ta.utils import dropna

In [22]:
# Use btcusd_test again:

btcusd_test3 = btcusd_test.copy()

In [23]:
btcusd_test3.head(2)

,open,high,low,close,volume
time,,,,,
2020-12-26 00:00:00,24704.71,24770.3,24515.00,24600.00,571.509687
2020-12-26 01:00:00,24595.01,24900.0,24490.01,24895.46,732.313376


In [24]:
btcusd_pdta = add_all_ta_features(btcusd_test3, open="open", high="high", low="low",
                                  close="close", volume="volume", fillna=True)

/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [25]:
# Slightly easier to interpret as it adds category to the features automatically

btcusd_pdta.head(2)

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-12-26 00:00:00,24704.71,24770.3,24515.00,24600.00,571.509687,-190.950945,571.509687,-0.334117,0.00000,50.0,0.000000e+00,0.000000e+00,-154.146947,1000.0,24628.433333,0.0,24600.00,24600.00,24600.00,0.000000,0.00,0.0,0.0,24628.433333,24883.733333,24373.133333,2.073213,0.444314,0.0,0.0,24515.00,24770.3,24642.650,1.037805,0.332942,0.0,0.000000,0.000000,0.000000,24600.00,24600.00,24600.000000,24600.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-26.385206,1.000000,0.000000,9089.793826,-269.808532,-269.808532,0.000000,24642.650,24642.650,24642.650,24642.650,33443.684787,33188.380743,4.0,4.0,0.0,-1.0,-1.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.000000,33.294164,33.294164,-66.705836,0.0,24600.000000,0.0,0.000000,0.000000,0.000000,-26.980853,0.000000,0.000000
2020-12-26 01:00:00,24595.01,24900.0,24490.01,24895.46,732.313376,525.143973,1303.823063,0.402772,216369.31002,100.0,2.931126e+09,2.931126e+09,-145.402690,1000.0,24703.353991,0.0,24747.73,25043.19,24452.27,2.387775,0.75,0.0,0.0,24695.128333,25027.773333,24362.483333,2.694013,0.801119,0.0,0.0,24490.01,24900.0,24695.005,1.656677,0.988927,0.0,23.569459,4.713892,18.855567,24747.73,24747.73,24645.455385,24621.885926,0.0,0.0,0.0,0.040168,0.029243,0.010925,0.002346,2.094652,66.666667,8942.063826,-265.423525,-267.616029,2.192504,24695.005,24695.005,24695.005,24695.005,33443.684787,33188.380743,8.0,4.0,4.0,-1.0,-1.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,60.943348,98.892656,66.093410,-1.107344,0.0,24707.701934,0.0,2.198694,0.439739,1.758955,1.201057,1.193901,1.201057


In [26]:
# Slight discrepancies between n close & n+1 open --> ok if they're not exact??

btcusd_pdta.tail(2)

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-02-05 23:00:00,37832.09,38336.31,37808.0,38311.49,605.383873,169962.019290,44619.653164,-0.004482,24494.758859,55.749448,1.835259e+10,3.017610e+09,7.234676,1977.034014,37886.740685,413.545707,37735.7115,38390.635031,37080.787969,3.471107,0.939577,0.0,0.0,37874.517333,38248.447333,37500.587333,1.974573,1.084297,1.0,0.0,37030.56,38349.00,37689.780,3.493879,0.971550,1.149118,167.620528,151.090377,16.530152,37916.209167,37579.415385,37846.039317,37678.418789,11.833867,22.299398,17.149998,1.099007,0.904044,0.194962,0.055817,23.338516,116.054431,164.2585,13.561217,10.635347,2.925870,37812.000,37477.435,37644.7175,37462.725,37230.975,36797.28,60.0,12.0,48.0,37275.0000,38207.421324,1.0,0.0,12.919044,60.590214,0.828486,0.553153,0.431063,9.966650,59.299585,96.619442,68.692058,-3.380558,306.397912,37625.568865,1.574328,-11.548570,-8.437690,-3.110880,1.265652,1.257710,55.737764
2021-02-06 00:00:00,38327.09,38872.95,38250.0,38761.47,1432.846100,170882.035522,46052.499264,0.078939,113102.948761,59.096908,2.127388e+10,4.929537e+09,24.491265,1977.034014,37993.600226,434.486137,37809.1980,38569.683194,37048.712806,4.022752,1.126095,1.0,0.0,37919.567667,38324.732667,37514.402667,2.136971,1.538962,1.0,0.0,37030.56,38872.95,37951.755,4.872862,0.939492,1.128577,228.230002,166.518302,61.711700,37988.000833,37635.625769,37986.874807,37758.644805,13.127328,28.647071,15.444906,1.174705,0.793142,0.381562,0.061264,23.467968,206.662076,519.6820,15.331694,11.451483,3.880211,38073.975,37739.410,37906.6925,37514.700,37230.975,36960.25,100.0,8.0,92.0,37296.2262,38207.421324,0.0,0.0,33.975780,65.668657,1.000000,0.742630,0.563840,13.387192,62.525109,93.023561,81.036093,-6.976439,481.517294,37680.571420,2.273089,-3.525574,-7.455267,3.929693,1.174530,1.167686,57.566951


In [27]:
# 88 cols
# 83 TI cols

len(list(btcusd_pdta.columns))

88

In [28]:
btcusd_pdta.isnull().sum()

open                   0
high                   0
low                    0
close                  0
volume                 0
                      ..
momentum_ppo_signal    0
momentum_ppo_hist      0
others_dr              0
others_dlr             0
others_cr              0
Length: 88, dtype: int64

In [29]:
btcusd_pdta.describe()

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
count,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1.009000e+03,1009.000000,1.009000e+03,1.009000e+03,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000
mean,33675.762151,34000.029822,33306.742587,33689.793826,1398.405413,106825.663101,42345.367522,0.101424,-3.103134e+03,52.680297,3.913119e+08,3.643381e+08,0.085859,1583.565722,33542.457546,689.449202,33565.827734,34886.831879,32244.823589,7.788661,0.560176,0.078295,0.050545,33606.359603,34298.963986,32913.755220,4.084152,0.598869,0.194252,0.115956,31875.404480,34959.766224,33417.585352,9.100066,0.599441,2.680689,90.044845,89.384713,0.660132,33617.462516,33528.290918,33616.822754,33526.777909,25.367747,19.323378,20.721768,1.023200,0.953445,0.069755,0.015823,24.710321,20.575605,77.445380,3.332103,2.222578,1.109524,33523.178915,33364.190659,33443.684787,33188.380743,33340.492745,33083.269390,55.028741,40.317146,14.711596,32927.544828,33861.992696,0.044599,0.044599,47.974089,53.519010,0.519319,0.518052,0.516745,8.350642,55.189135,59.182040,59.132491,-40.817960,183.298560,33658.311457,0.590740,-1.625277,-1.595722,-0.029555,0.027022,0.045062,36.950381
std,3782.537268,3827.323228,3724.276025,3775.334723,1151.263681,52210.411496,19291.273744,0.134927,3.790774e+05,18.236421,2.163223e+10,5.760809e+09,61.747335,339.208990,3755.153004,297.253927,3772.926193,4077.387021,3602.254808,4.245694,0.324748,0.268769,0.219175,3767.757166,3901.447157,3658.514923,1.820288,0.492606,0.395820,0.320331,3559.883919,4046.227792,3731.779349,4.308014,0.264164,2.108904,458.405924,431.451504,138.545523,3770.847130,3773.107531,3751.300852,3731.931234,11.161383,7.251536,7.748540,0.180423,0.174018,0.286149,0.852420,2.680762,107.777566,1091.112549,58.650381,60.166801,15.773660,3742.026994,3711.011322,3708.307206,3611.643409,3650.537317,3550.087751,34.269592,33.419

## 1. Scraping for 5 years of hourly BTC-USD Data:

In [30]:
# btcusd_df = get_crypto_df('BTC-USD', delta=38)

In [31]:
# # time to index:

# btcusd_df.set_index("time", inplace=True)

In [32]:
# # check
# btcusd_df.head()

In [33]:
# btcusd_df.tail()

In [34]:
# # describe
# btcusd_df.describe()

In [35]:
# Save to csv (OHLCV):
# btcusd_df.to_csv('BTCUSD.csv')

## 2. Scraping for 5 years of hourly ETH-USD Data:

In [36]:
# ethusd_df = get_crypto_df('ETH-USD', delta=38)

In [37]:
# ethusd_df.set_index("time", inplace=True)

In [38]:
# ethusd_df.head()

In [39]:
# ethusd_df.tail()

In [40]:
# ethusd_df.describe()

In [41]:
# ethusd_df.to_csv('ETHUSD.csv')

## 3. Scraping for 5 years of hourly LTC-USD Data:

In [42]:
# ltcusd_df = get_crypto_df('LTC-USD', delta=38)

In [43]:
# ltcusd_df.set_index("time", inplace=True)

In [44]:
# ltcusd_df.head()

In [45]:
# ltcusd_df.tail()

In [46]:
# ltcusd_df.describe()

In [47]:
# ltcusd_df.to_csv('LTCUSD.csv')

## 4. Scraping for max possible amount of hourly XLM-USD Data:

In [48]:
# xlmusd_df = get_crypto_df('XLM-USD', delta=38)

In [49]:
# xlmusd_df.set_index("time", inplace=True)

In [50]:
# xlmusd_df.head()

In [51]:
# xlmusd_df.tail()

In [52]:
# xlmusd_df.describe()

In [53]:
# Limited to Q1 2019

# xlmusd_df.to_csv('XLMUSD.csv')

## 5. Scraping for max possible amount of hourly DASH-USD Data:

In [54]:
# dashusd_df = get_crypto_df('DASH-USD', delta=38)

In [55]:
# dashusd_df.set_index("time", inplace=True)

In [56]:
# dashusd_df.head()

In [57]:
# dashusd_df.tail()

In [58]:
# dashusd_df.describe()

In [59]:
# Sadly only limited to Q3 2019

# dashusd_df.to_csv('DASHUSD.csv')

## 6. Scraping the max possible amount of LINK

In [60]:
# # Test

# linkusd_df = get_crypto_df('LINK-USD', delta=38)

In [61]:
# linkusd_df.set_index("time", inplace=True)

In [62]:
# linkusd_df.head()

In [63]:
# linkusd_df.tail()

In [64]:
# linkusd_df.describe()

In [65]:
# linkusd_df.to_csv('LINKUSD.csv')

## Comment

- First 3 cryptocurrencies (BTC, ETH & LTC) have more data (dating back to 2016).
- Latter 3 (XLM, DASH & LINK) only have data dating back to mid 2019 (so ~18 months)

- Data without TI's can be used for EDA stage
- Add TI's when you want to do modeling in Phase 4

## Load from csvs again
- Check that csv sizes not too large

- df = df.sort_index(ascending=True, axis=0)
- datetime index: df.index = pd.to_datetime(df.index)

In [62]:
# Re use this for each crypto df below

def df_time_cln(df):
    
    """ Quickly cleans your time series df, converting index to a datetime
        format and sorting it chronologically. Loading from csv will create a
        new index column so you have to run this everytime after importing. """
    
    df1 = df.copy()
    
    df1.index = df1['time']
    df1.drop(columns='time', inplace=True)
    df1.sort_index(inplace=True) # Sort it chronologically
    df1.index = pd.to_datetime(df1.index)
    
    return df1

# Create technical indicators for all 6 cryptocurrencies

- Don't overwrite the current OHLCV csvs - save as a new copy

#### How to treat missing values in a time series?
Sometimes, your time series will have missing dates/times. That means, the data was not captured or was not available for those periods. It could so happen the measurement was zero on those days, in which case, case you may fill up those periods with zero.

Secondly, when it comes to time series, **you should typically NOT replace missing values with the mean of the series, especially if the series is not stationary**. What you could do instead for a quick and dirty workaround is to **forward-fill the previous value.**

However, depending on the nature of the series, you want to try out multiple approaches before concluding. Some effective alternatives to imputation are:

- Backward Fill
- Linear Interpolation
- Quadratic interpolation
- Mean of nearest neighbors
- Mean of seasonal couterparts

To measure the imputation performance, you can manually introduce missing values to the time series (those with actual true values), impute it with above approaches and then measure the mean squared error of the imputed against the actual values.

## Save as new csvs (separate files for each crypto)


- These files too large to git push - send to postgres or gitignore
- One copy (cleaned) with only OHLCV - Phase 3 EDA & ARIMA model
- Another copy (with TIs) for modeling

### BTC

In [60]:
# BTC

btc = pd.read_csv('/Users/marzimin/Downloads/crypto_data/BTCUSD.csv')
btc.head(3)

,time,open,high,low,close,volume
0,2016-01-03 00:00:00,435.40,435.75,435.08,435.40,120.975708
1,2016-01-03 01:00:00,435.39,435.70,433.00,433.52,360.387820
2,2016-01-03 02:00:00,433.51,433.52,432.07,433.43,154.767890


In [63]:
# Cleaned

btc_cln = df_time_cln(btc)
btc_cln.head(3)

,open,high,low,close,volume
time,,,,,
2016-01-03 00:00:00,435.40,435.75,435.08,435.40,120.975708
2016-01-03 01:00:00,435.39,435.70,433.00,433.52,360.387820
2016-01-03 02:00:00,433.51,433.52,432.07,433.43,154.767890


In [69]:
# btc_ta = add_all_ta_features(
#     btc_cln, open="open", high="high", low="low",
#     close="close", volume="volume", fillna=True)

/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [70]:
# btc_ta.head(3)

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-03 00:00:00,435.40,435.75,435.08,435.40,120.975708,-5.416823,120.975708,-0.044776,0.000000,50.0,0.000000e+00,0.000000e+00,-113.160441,1000.000000,435.410000,0.0,435.400000,435.400000,435.400000,0.000000,0.000000,0.0,0.0,435.410000,436.080000,434.740000,0.307756,0.492537,0.0,0.0,435.08,435.75,435.415,0.153881,0.477612,0.0,0.000000,0.000000,0.000000,435.400000,435.400000,435.400000,435.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-93.354748,1.000000,0.000000,6132.633577,-933.709230,-933.709230,0.000000,435.415,435.415,435.415,435.415,6549.847741,6530.430297,4.0,4.0,0.0,-1.0,-1.00,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.000000,47.761194,47.761194,-52.238806,0.0,435.400000,0.0,0.00000,0.000000,0.000000,-93.370923,0.000000,0.000000
2016-01-03 01:00:00,435.39,435.70,433.00,433.52,360.387820,-226.988593,-239.412112,-0.471553,-677.529101,0.0,-7.978905e+05,-7.978905e+05,-114.512242,1000.000000,434.409263,0.0,434.460000,436.340000,432.580000,0.865442,0.250000,0.0,0.0,434.741667,436.426667,433.056667,0.775173,0.137488,0.0,0.0,433.00,435.75,434.375,0.632970,0.189091,0.0,-0.149972,-0.029994,-0.119977,434.460000,434.460000,435.110769,435.260741,0.0,0.0,0.0,0.000101,0.000448,-0.000347,-0.000843,2.432375,-66.666667,6133.573577,-933.852348,-933.780789,-0.071559,434.375,434.375,434.375,434.375,6549.847741,6530.430297,4.0,8.0,-4.0,-1.0,-1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-100.0,15.430267,18.909091,33.335142,-81.090909,0.0,434.570762,0.0,13.76859,2.753718,11.014872,-0.431787,-0.432722,-0.431787
2016-01-03 02:00:00,433.51,433.52,432.07,433.43,154.767890,-91.433269,-394.180002,-0.143733,-582.729102,0.0,-1.456859e+06,-1.127375e+06,-1.588238,999.792397,434.068018,0.0,434.116667,435.933061,432.300272,0.836823,0.310981,0.0,0.0,434.163333,435.770000,432.556667,0.740121,0.271784,0.0,0.0,432.07,435.75,433.910,0.847698,0.369565,0.0,-0.272941,-0.078584,-0.194357,434.116667,434.116667,434.852189,435.125130,0.0,0.0,0.0,0.000186,0.001040,-0.000854,-0.002254,3.817012,-92.780749,6133.916910,-933.904621,-933.822067,-0.082555,433.910,433.910,433.910,433.910,6549.847741,6530.430297,4.0,12.0,-8.0,-1.0,435.75,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-100.0,39.004149,36.956522,34.542269,-63.043478,0.0,434.059856,0.0,12.46693,4.696360,7.770570,-0.020760,-0.020762,-0.452458


### ETH

In [64]:
# ETH

eth = pd.read_csv('/Users/marzimin/Downloads/crypto_data/ETHUSD.csv')
eth_cln = df_time_cln(eth)
eth_cln.head(3)

,open,high,low,close,volume
time,,,,,
2016-05-18 00:00:00,12.5,14.93,12.5,13.0,8.002848
2016-05-18 01:00:00,14.0,14.10,14.0,14.1,15.462483
2016-05-18 02:00:00,14.0,14.00,14.0,14.0,0.052000


In [72]:
# eth_ta = add_all_ta_features(
#     eth_cln, open="open", high="high", low="low",
#     close="close", volume="volume", fillna=True)

/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


### LTC

In [73]:
# LTC

ltc = pd.read_csv('/Users/marzimin/Downloads/crypto_data/LTCUSD.csv')
ltc_cln = df_time_cln(ltc)
ltc_cln.head(3)

,open,high,low,close,volume
time,,,,,
2016-08-17 04:00:00,3.70,3.70,3.70,3.70,10.000000
2016-08-17 20:00:00,3.62,3.62,3.62,3.62,0.317691
2016-08-18 22:00:00,3.62,3.70,3.62,3.70,1.000000


In [74]:
# ltc_ta = add_all_ta_features(
#     ltc_cln, open="open", high="high", low="low",
#     close="close", volume="volume", fillna=True)

/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


### XLM

In [75]:
# XLM

xlm = pd.read_csv('/Users/marzimin/Downloads/crypto_data/XLMUSD.csv')
xlm_cln = df_time_cln(xlm)
xlm_cln.head(3)

,open,high,low,close,volume
time,,,,,
2019-03-14 16:00:00,0.122699,0.122699,0.1074,0.107498,61154
2019-03-14 17:00:00,0.107498,0.107499,0.1060,0.106550,262974
2019-03-14 18:00:00,0.106573,0.106573,0.1062,0.106298,61718


In [76]:
# xlm_ta = add_all_ta_features(
#     xlm_cln, open="open", high="high", low="low",
#     close="close", volume="volume", fillna=True)

/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


### DASH

In [77]:
# DASH

dash = pd.read_csv('/Users/marzimin/Downloads/crypto_data/DASHUSD.csv')
dash_cln = df_time_cln(dash)
dash_cln.head(3)

,open,high,low,close,volume
time,,,,,
2019-09-17 16:00:00,93.391,104.220,93.391,99.000,6805.455
2019-09-17 17:00:00,98.000,105.000,96.002,96.184,2893.866
2019-09-17 18:00:00,96.200,96.489,94.000,95.327,861.343


In [78]:
# dash_ta = add_all_ta_features(
#     dash_cln, open="open", high="high", low="low",
#     close="close", volume="volume", fillna=True)

/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


### LINK

In [79]:
# LINK

link = pd.read_csv('/Users/marzimin/Downloads/crypto_data/LINKUSD.csv')
link_cln = df_time_cln(link)
link_cln.head(3)

,open,high,low,close,volume
time,,,,,
2019-06-27 16:00:00,2.80099,2.80099,2.25,2.3809,18267.81
2019-06-27 17:00:00,2.35000,2.78000,2.20,2.3900,66843.27
2019-06-27 18:00:00,2.60000,2.61000,2.20,2.3389,67810.38


In [80]:
# link_ta = add_all_ta_features(
#     link_cln, open="open", high="high", low="low",
#     close="close", volume="volume", fillna=True)

/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/marzimin/opt/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


### Note: these files are large, can't git push - delete from directory

- Did df.to_csv(name.csv) for all datasets

- Moved to local downloads folder, read csv with new filepath

In [81]:
# Test
# Clean the time index too

btc_ta = pd.read_csv('/Users/marzimin/Downloads/crypto_data/BTC_TA.csv')

In [82]:
btc_ta.head(2)

,time,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,2016-01-03 00:00:00,435.40,435.75,435.08,435.40,120.975708,-5.416823,120.975708,-0.044776,0.000000,50.0,0.000000,0.000000,-113.160441,1000.0,435.410000,0.0,435.40,435.40,435.40,0.000000,0.00,0.0,0.0,435.410000,436.080000,434.740000,0.307756,0.492537,0.0,0.0,435.08,435.75,435.415,0.153881,0.477612,0.0,0.000000,0.000000,0.000000,435.40,435.40,435.400000,435.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-93.354748,1.000000,0.000000,6132.633577,-933.709230,-933.709230,0.000000,435.415,435.415,435.415,435.415,6549.847741,6530.430297,4.0,4.0,0.0,-1.0,-1.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.000000,47.761194,47.761194,-52.238806,0.0,435.400000,0.0,0.00000,0.000000,0.000000,-93.370923,0.000000,0.000000
1,2016-01-03 01:00:00,435.39,435.70,433.00,433.52,360.387820,-226.988593,-239.412112,-0.471553,-677.529101,0.0,-797890.451279,-797890.451279,-114.512242,1000.0,434.409263,0.0,434.46,436.34,432.58,0.865442,0.25,0.0,0.0,434.741667,436.426667,433.056667,0.775173,0.137488,0.0,0.0,433.00,435.75,434.375,0.632970,0.189091,0.0,-0.149972,-0.029994,-0.119977,434.46,434.46,435.110769,435.260741,0.0,0.0,0.0,0.000101,0.000448,-0.000347,-0.000843,2.432375,-66.666667,6133.573577,-933.852348,-933.780789,-0.071559,434.375,434.375,434.375,434.375,6549.847741,6530.430297,4.0,8.0,-4.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-100.0,15.430267,18.909091,33.335142,-81.090909,0.0,434.570762,0.0,13.76859,2.753718,11.014872,-0.431787,-0.432722,-0.431787


In [83]:
eth_ta = pd.read_csv('/Users/marzimin/Downloads/crypto_data/ETH_TA.csv')

In [84]:
ltc_ta = pd.read_csv('/Users/marzimin/Downloads/crypto_data/LTC_TA.csv')

In [85]:
xlm_ta = pd.read_csv('/Users/marzimin/Downloads/crypto_data/XLM_TA.csv')

In [86]:
dash_ta = pd.read_csv('/Users/marzimin/Downloads/crypto_data/DASH_TA.csv')

In [87]:
link_ta = pd.read_csv('/Users/marzimin/Downloads/crypto_data/LINK_TA.csv')

### Saving it as CSVs are neat, but it would be better to send them to a postgres database:

In [65]:
import psycopg2
from sqlalchemy import create_engine

# DSN (data source name) format for database connections:  
# [protocol / database  name]://[username]:[password]@[hostname / ip]:[port]/[database name here]


# on your computer you are the user postgres (full administrative access)
# Will be different (provided) for remote servers
db_user = 'postgres'

# if you need a password to access a database, put it here
# Passwords will be required if you're accessing a remote server
db_password = ''

# on your computer, use localhost
# will be different for remote servers
db_host = 'localhost'

# the default port for postgres is 5432
db_port = 5432

# we want to connect to a standard database
# check your postgres app for available local databases
database = 'marzimin'

# Code syntax to create the database:

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}'      
engine = create_engine(conn_str)
conn = engine.connect()
conn.execute("commit")

try:
    conn.execute(f'create database {database}')
except:
    print('database already exists')

conn.close()

database already exists


### Export all datasets to postgres database

In [66]:
pathData = '/Users/marzimin/Downloads/crypto_data/'

coin_tables = !ls $pathData
coin_tables

['BTCUSD.csv',
 'BTC_TA.csv',
 'DASHUSD.csv',
 'DASH_TA.csv',
 'ETHUSD.csv',
 'ETH_TA.csv',
 'LINKUSD.csv',
 'LINK_TA.csv',
 'LTCUSD.csv',
 'LTC_TA.csv',
 'XLMUSD.csv',
 'XLM_TA.csv']

In [67]:
engine = create_engine(conn_str+f'/{database}')

for table in coin_tables:
    table_name = table.split('.')[0] # gets rid of .csv
    print(table_name)
    
    try:
        df = pd.read_csv(pathData + table) # Add full path
        display(df.head(2))
        df.to_sql(table_name, engine, index=False, if_exists='replace') # sends it to my database
    except:
        print(f'No values for table {table_name}.')

BTCUSD


,time,open,high,low,close,volume
0,2016-01-03 00:00:00,435.40,435.75,435.08,435.40,120.975708
1,2016-01-03 01:00:00,435.39,435.70,433.00,433.52,360.387820


BTC_TA


,time,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,03/01/16 00:00,435.40,435.75,435.08,435.40,120.975708,-5.416823,120.975708,-0.044776,0.000000,50.0,0.0000,0.0000,-113.160441,1000.0,435.410000,0.0,435.40,435.40,435.40,0.000000,0.00,0,0,435.410000,436.080000,434.740000,0.307756,0.492537,0,0,435.08,435.75,435.415,0.153881,0.477612,0.0,0.000000,0.000000,0.000000,435.40,435.40,435.400000,435.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-93.354748,1.000000,0.000000,6132.633577,-933.709230,-933.709230,0.000000,435.415,435.415,435.415,435.415,6549.847741,6530.430297,4,4,0,-1.0,-1.0,0,0,0.0,100.0,0.0,0.0,0.0,0.0,0.000000,47.761194,47.761194,-52.238806,0.0,435.400000,0.0,0.00000,0.000000,0.000000,-93.370923,0.000000,0.000000
1,03/01/16 01:00,435.39,435.70,433.00,433.52,360.387820,-226.988593,-239.412112,-0.471553,-677.529101,0.0,-797890.4513,-797890.4513,-114.512242,1000.0,434.409263,0.0,434.46,436.34,432.58,0.865442,0.25,0,0,434.741667,436.426667,433.056667,0.775173,0.137488,0,0,433.00,435.75,434.375,0.632970,0.189091,0.0,-0.149972,-0.029994,-0.119977,434.46,434.46,435.110769,435.260741,0.0,0.0,0.0,0.000101,0.000448,-0.000347,-0.000843,2.432375,-66.666667,6133.573577,-933.852348,-933.780789,-0.071559,434.375,434.375,434.375,434.375,6549.847741,6530.430297,4,8,-4,-1.0,-1.0,0,0,0.0,0.0,0.0,0.0,0.0,-100.0,15.430267,18.909091,33.335142,-81.090909,0.0,434.570762,0.0,13.76859,2.753718,11.014872,-0.431787,-0.432722,-0.431787


DASHUSD


...

,time,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,17/08/16 04:00,3.70,3.70,3.70,3.70,10.000000,0.0,10.000000,0.0,0.000000,50.0,0.0,0.0,16.612203,1000.000000,3.700000,0.0,3.70,3.70,3.70,0.000000,0.00,0,0,3.70,3.70,3.70,0.0,0.0,0,1,3.70,3.7,3.70,0.000000,0.0,0.0,0.000000,0.000000,0.000000,3.70,3.70,3.700000,3.700000,0.0,0.0,0.0,0.000000,0.000000,0.0,-94.526927,0.0,0.000000,63.974132,-945.326229,-945.326229,0.000000,3.70,3.70,3.70,3.70,67.559564,67.486964,4,4,0,-1.0,-1.0,0,0,0.0,100.0,0.0,0.0,0.0,0.0,50.0,50.0,50.0,-50.0,0.0,3.70000,0.0,0.000000,0.000000,0.000000,-94.532623,0.000000,0.000000
1,17/08/16 20:00,3.62,3.62,3.62,3.62,0.317691,0.0,9.682309,0.0,-0.025415,0.0,0.0,0.0,-9.460131,978.378378,3.697537,0.0,3.66,3.74,3.58,4.371585,0.25,0,0,3.66,3.66,3.66,0.0,0.0,0,1,3.62,3.7,3.66,2.185792,0.0,0.0,-0.006382,-0.001276,-0.005105,3.66,3.66,3.687692,3.694074,0.0,0.0,0.0,0.001249,0.001249,0.0,-0.004223,0.0,-66.666667,64.014132,-945.917297,-945.621763,-0.295534,3.66,3.66,3.66,3.66,67.559564,67.486964,4,8,-4,-1.0,-1.0,0,0,0.0,0.0,0.0,0.0,0.0,-100.0,0.0,0.0,0.0,-100.0,0.0,3.66238,0.0,-8.320535,-1.664107,-6.656428,-2.162162,-2.185879,-2.162162


XLMUSD


,time,open,high,low,close,volume
0,2019-03-14 16:00:00,0.122699,0.122699,0.1074,0.107498,61154
1,2019-03-14 17:00:00,0.107498,0.107499,0.1060,0.106550,262974


XLM_TA


,time,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,14/03/19 16:00,0.122699,0.122699,0.1074,0.107498,61154,-60370.53807,61154,-0.987189,0.000000,50.0,0.000000,0.000000,23681.062640,1000.0,0.112532,0.0,0.107498,0.107498,0.107498,0.000000,0.00,0,0,0.112532,0.127831,0.097233,27.190407,0.335469,0,0,0.1074,0.122699,0.115049,14.231893,0.006406,0.0,0.000000,0.000000,0.000000,0.107498,0.107498,0.107498,0.107498,0.0,0.0,0.0,0.000000,0.000000,0.000000,17.310524,1.000000,0.000000,-0.015599,169.737434,169.737434,0.000000,0.115049,0.115049,0.115049,0.115049,0.091909,0.091906,4,4,0,-1.0,-1.0,0,0,0.0,100.0,0.0,0.0,0.0,0.0,0.000000,0.640565,0.640565,-99.359435,0.0,0.107498,0.0,0.00000,0.000000,0.000000,16.973743,0.000000,0.000000
1,14/03/19 17:00,0.107498,0.107499,0.1060,0.106550,262974,-130368.28720,-201820,-0.402212,-249.299352,0.0,-0.004731,-0.004731,8061.016168,1000.0,0.107787,0.0,0.107024,0.107972,0.106076,1.771565,0.25,0,0,0.109608,0.118007,0.101209,15.325570,0.317974,0,0,0.1060,0.122699,0.114350,15.603042,0.032936,0.0,-0.000076,-0.000015,-0.000061,0.107024,0.107024,0.107352,0.107428,0.0,0.0,0.0,0.003065,0.517017,-0.513952,-0.001722,1.850275,-66.666667,-0.015125,164.579612,167.158523,-2.578911,0.114350,0.114350,0.114350,0.114350,0.091909,0.091906,4,8,-4,-1.0,-1.0,0,0,0.0,0.0,0.0,0.0,0.0,-100.0,3.274199,3.293610,1.967088,-96.706390,0.0,0.106941,0.0,21.15484,4.230968,16.923872,-0.881877,-0.885788,-0.881877


In [55]:
# All tables in 'marzimin' database

engine.table_names()

['BTC_TA',
 'DASH_TA',
 'ETH_TA',
 'LINK_TA',
 'LTC_TA',
 'XLM_TA',
 'BTCUSD',
 'DASHUSD',
 'ETHUSD',
 'LINKUSD',
 'LTCUSD',
 'XLMUSD']

In [68]:
# You'd have to know the terminology e.g. tablename, the catalog, etc.
# Below a default method that can be used to read tables from any of your databases

query = f"""
      SELECT tablename
      FROM pg_catalog.pg_tables
      WHERE schemaname != 'pg_catalog' AND 
      schemaname != 'information_schema'
         ;"""
pd.read_sql(query, engine)

,tablename
0,BTCUSD
1,BTC_TA
2,DASHUSD
3,DASH_TA
4,ETHUSD
5,ETH_TA
6,LINKUSD
7,LINK_TA
8,LTCUSD
9,LTC_TA


In [69]:
# Now you can query
# Add double quotes to call the table name
# Note the new index - so you'll have to fix that too in your script

query = f"""
      SELECT * from "BTC_TA"
      LIMIT 5
         ;"""
pd.read_sql(query, engine)

,time,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,03/01/16 00:00,435.40,435.75,435.08,435.40,120.975708,-5.416823,120.975708,-0.044776,0.000000,50.000000,0.000000e+00,0.000000e+00,-113.160441,1000.000000,435.410000,0.0,435.400000,435.400000,435.400000,0.000000,0.000000,0,0,435.410000,436.080000,434.740000,0.307756,0.492537,0,0,435.08,435.75,435.415,0.153881,0.477612,0.0,0.000000,0.000000,0.000000,435.400000,435.400000,435.400000,435.400000,0.0,0.0,0.0,0.000000,0.000000,0.000000,-93.354748,1.000000,0.000000,6132.633577,-933.709230,-933.709230,0.000000,435.415,435.415,435.415,435.415,6549.847741,6530.430297,4,4,0,-1.0,-1.0000,0,0,0.0,100.000000,0.0,0.0,0.0,0.000000,0.000000,47.761194,47.761194,-52.238806,0.0,435.400000,0.0,0.000000,0.000000,0.000000,-93.370923,0.000000,0.000000
1,03/01/16 01:00,435.39,435.70,433.00,433.52,360.387820,-226.988593,-239.412112,-0.471553,-677.529101,0.000000,-7.978905e+05,-7.978905e+05,-114.512242,1000.000000,434.409263,0.0,434.460000,436.340000,432.580000,0.865442,0.250000,0,0,434.741667,436.426667,433.056667,0.775173,0.137488,0,0,433.00,435.75,434.375,0.632970,0.189091,0.0,-0.149972,-0.029994,-0.119977,434.460000,434.460000,435.110769,435.260741,0.0,0.0,0.0,0.000101,0.000448,-0.000347,-0.000843,2.432375,-66.666667,6133.573577,-933.852348,-933.780789,-0.071559,434.375,434.375,434.375,434.375,6549.847741,6530.430297,4,8,-4,-1.0,-1.0000,0,0,0.0,0.000000,0.0,0.0,0.0,-100.000000,15.430267,18.909091,33.335142,-81.090909,0.0,434.570762,0.0,13.768590,2.753718,11.014872,-0.431787,-0.432722,-0.431787
2,03/01/16 02:00,433.51,433.52,432.07,433.43,154.767889,-91.433269,-394.180002,-0.143733,-582.729102,0.000000,-1.456859e+06,-1.127375e+06,-1.588238,999.792397,434.068017,0.0,434.116667,435.933061,432.300272,0.836823,0.310981,0,0,434.163333,435.770000,432.556667,0.740121,0.271784,0,0,432.07,435.75,433.910,0.847698,0.369565,0.0,-0.272941,-0.078584,-0.194357,434.116667,434.116667,434.852189,435.125130,0.0,0.0,0.0,0.000186,0.001040,-0.000854,-0.002254,3.817012,-92.780749,6133.916910,-933.904621,-933.822067,-0.082555,433.910,433.910,433.910,433.910,6549.847741,6530.430297,4,12,-8,-1.0,435.7500,0,1,0.0,0.000000,0.0,0.0,0.0,-100.000000,39.004149,36.956522,34.542269,-63.043478,0.0,434.059856,0.0,12.466930,4.696360,7.770570,-0.020760,-0.020762,-0.452458
3,03/01/16 03:00,433.48,434.00,432.95,434.00,87.075675,-4.357594,-307.104326,-0.006025,-492.391640,14.455907,8.199764e+05,-4.782577e+05,0.082382,1001.107215,434.017687,0.0,434.087500,435.663785,432.511215,0.726252,0.472245,0,0,434.035000,435.502500,432.567500,0.676213,0.488075,0,0,432.07,435.75,433.910,0.847755,0.524457,0.0,-0.320704,-0.127008,-0.193696,434.087500,434.087500,434.721083,435.041787,0.0,0.0,0.0,0.000500,0.001132,-0.000632,-0.003724,5.085935,-36

In [59]:
# Read it as your data
# Just remove the LIMIT 5 to get the whole dataset

# btc_ta = pd.read_sql(query,engine)
# btc_ta.info()